## Tex Preamble
$$
\newcommand{\bm}[1]{{\bf #1}}
\newcommand{\eq}[1]{Eq.(\ref{eq:#1})}
\newcommand{\zu}[1]{図\ref{fig:#1}}
\newcommand{\shiki}[1]{式(\ref{eq:#1})}
\newcommand{\fig}[1]{Fig.\ \ref{fig:#1}}
\newcommand{\grp}[1]{Fig.\ \ref{grp:#1}}
\newcommand{\tab}[1]{Table\ \ref{tab:#1}}
\newcommand{\defeq}{:=}
\newcommand{\tvec}[1]{\bvec{#1}^{\mathsf{T}}} 
\newcommand{\mr}[1]{\mathrm{#1}}%rm体
\newcommand{\ml}[1]{\mathcal{#1}}%frac体
\newcommand{\bmat}[1]{\begin{bmatrix} #1 \end{bmatrix}}%行列簡略化
\newcommand{\pmat}[1]{\begin{pmatrix} #1 \end{pmatrix}}%行列簡略化
\newcommand{\mat}[1]{\left( \begin{matrix} #1 \end{matrix} \right)}%行列簡略化
%\newcommand{\nom}[1]{ \ml{N} \{ 0,\sigma_{ #1 } \} }
\newcommand{\expo}[1]{ \mathrm{e}^{#1} }
$$

Texプリアンブル＋Yalmipの初期設定記述箇所。

addpathでSeDumiやSDpt3などのソルバをインストール。
PENLABもインストールした。

In [1]:
addpath(genpath('C:\Users\hkedw\Documents\MATLAB\SeDuMi_1_3'))
addpath(genpath('C:\Users\hkedw\Documents\MATLAB\SDpt3-4.0'))
addpath(genpath('C:\Users\hkedw\Documents\MATLAB\YALMIP-master'))
addpath(genpath('C:\Users\hkedw\Documents\MATLAB\PENLABv104'))


In [3]:
% ----------
J  = 0.0712;  M  = 0.390;  mu = 0.695;
l  = 0.204;   g  = 9.81;

A = [     0      1
      M*g*l/J  -mu/J ];
B = [  0
      1/J ];

rank(ctrb(A,B))


ans =

     2




## アルファ領域（収束速度制約）

先程の式を展開すると以下のような制約をとけば良い。

$$
PA+A^\top P+2\alpha P \prec 0, P\succ0
$$

これは$X:=P^{-1}$を両辺からかけることで等価的に

$$
AX+XA^\top +2\alpha X \prec 0, X\succ0
$$

と表せる。


### BMIからLMIへの変換を用いた 設計問題
ここで以下のゲインKの状態FBシステムを安定化させることを考える。
$$
\dot{x}=Ax+Bu=(A+BK)x
$$

先ほどの式に代入して，（このままだとBMI：双線形行列不等式になるので）
$$
(A+BK)X+X(A+BK)^\top +2\alpha X \prec 0, X\succ0
$$

ここで，$KX=Y$と変換することで，LMIに再び変換することができる。

$$
(AX+BY)+(AX+BY)^\top +2\alpha X \prec 0, X\succ0
$$


ところがどっこい，これはBMIのままでも解けるっぽい。
- 普通にLMIで解いてみる

In [4]:
alpha = 5;

row=size(A,1);col=size(A,2);


X = sdpvar(row,col,'symmetric')
Y = sdpvar(1,col,'full')

LMI = [A*X+B*Y+(A*X+B*Y)'+ 2*alpha*X<0 ,X>0];

sol = solvesdp(LMI,Y*Y.');

X = double(X);
Y = double(Y);
K = Y/X;

% LMI1 result
pole1 = eig(A+B*K);

display(pole1);

Linear matrix variable 2x2 (symmetric, real, 3 variables)
Linear matrix variable 1x2 (full, real, 2 variables)
警告: Strict inequalities are not supported. A non-strict has been added instead')
> In < (line 12)
警告: Strict inequalities are not supported. A non-strict has been added instead')
> In > (line 12)
SeDuMi 1.3 by AdvOL, 2005-2008 and Jos F. Sturm, 1998-2003.
Alg = 2: xz-corrector, theta = 0.250, beta = 0.500
eqs m = 6, order n = 7, dim = 13, blocks = 4
nnz(A) = 16 + 0, nnz(ADA) = 30, nnz(L) = 18
 it :     b*y       gap    delta  rate   t/tP*  t/tD*   feas cg cg  prec
  0 :            1.51E+02 0.000
  1 :  -2.79E-01 1.40E+01 0.000 0.0928 0.9900 0.9900   1.54  1  1  4.9E+00
  2 :  -1.70E-02 3.10E+00 0.000 0.2213 0.9000 0.9000   1.36  1  1  1.6E+00
  3 :   1.07E-03 8.88E-01 0.000 0.2862 0.9000 0.9000   0.66  1  1  1.2E+00
  4 :  -2.91E-03 2.51E-01 0.000 0.2831 0.9000 0.9000   0.82  1  1  1.3E+00
  5 :  -5.27E-04 5.03E-02 0.000 0.2002 0.9000 0.9000   1.19  1  1  9.4E-01
  6 :  -1.26E

- BMIで解いてみる

sdp = 半正定値計画問題，なのでBMIはsdpでは解けないよね？→そうでもなさそう
PENLABを使ってみたが解けていると言っているのに解けていなかったりするのでよくわからん…


In [10]:
alpha = 5;

row=size(A,1);col=size(A,2);


X = sdpvar(row,col,'symmetric')
K = sdpvar(1,col,'full')

BMI = [(A+B*K)*X+X.'*(A+B*K).'+ 2*alpha*X <=0 ,X>eye(row)];
Objective = (K-[1,1])*(K-[1,1]).';
options = sdpsettings('solver','penlab');
sol = optimize(BMI, Objective, options);
sol.info
%sol = solvesdp(,);


K=double(K);
X=double(X);

BMI_val=(A+B*K)*X+X*(A+B*K)'+ 2*alpha*X;

display(eig(A+B*K))


Linear matrix variable 2x2 (symmetric, real, 3 variables)
Linear matrix variable 1x2 (full, real, 2 variables)
警告: Strict inequalities are not supported. A non-strict has been added instead')
> In > (line 12)
*******************************************************************************
PenLab 1.04 (20140125)
*******************************************************************************
Number of variables                            5
Number of matrix variables                     0
   - degrees of freedom (var. elements)        0
(Function) constraints
   - box inequalities                          0
   - linear inequalities                       0
   - nonlinear inequalities                    0
   - linear equalities                         0
   - nonlinear equalities                      0
Matrix constraints
   - box inequalities                          0
   - linear inequalities                       1
   - nonlinear inequalities                    1
 
*************************

In [12]:
alpha = 5;

row=size(A,1);col=size(A,2);


X = sdpvar(row,col,'symmetric')
K = sdpvar(1,col,'full')

BMI = [(A+B*K)*X+X.'*(A+B*K).'+ 2*alpha*X <=0 ,X>eye(row)];
Objective = (K-[0,1])*(K-[0,1]).';
options = sdpsettings('solver','penlab');
sol = optimize(BMI, Objective, options);
sol.info
%sol = solvesdp(,);


K=double(K);
X=double(X);

BMI_val=(A+B*K)*X+X*(A+B*K)'+ 2*alpha*X;
eig(BMI_val)

display(eig(A+B*K))

Linear matrix variable 2x2 (symmetric, real, 3 variables)
Linear matrix variable 1x2 (full, real, 2 variables)
警告: Strict inequalities are not supported. A non-strict has been added instead')
> In > (line 12)
*******************************************************************************
PenLab 1.04 (20140125)
*******************************************************************************
Number of variables                            5
Number of matrix variables                     0
   - degrees of freedom (var. elements)        0
(Function) constraints
   - box inequalities                          0
   - linear inequalities                       0
   - nonlinear inequalities                    0
   - linear equalities                         0
   - nonlinear equalities                      0
Matrix constraints
   - box inequalities                          0
   - linear inequalities                       1
   - nonlinear inequalities                    1
 
*************************

In [15]:
n = row;

% Wanted
K0 = ones(1,n);

X = sdpvar(n);
K = sdpvar(1,n);
% Some initial guess, problem dependent of course
assign(X,eye(n));
L = -lqr(A,B,eye(n),1);
assign(K,L);

S = X*(A + B*K) + (A+B*K)'*X + value(X);
ops = sdpsettings('verbose',0);
for i = 1:5
    Model = [reshape(linearize(S(:)),2,2) <= 0, X >= eye(n)];
    optimize(Model, norm(K-K0),ops)
    norm(K-K0)
end

display(double(K))


ans = 

  フィールドをもつ struct:

    yalmiptime: 0.1320
    solvertime: 0.0300
          info: 'Successfully solved (SeDuMi-1.3)'
       problem: 0

Nonlinear scalar (real, models 'norm', 1 variable, current value : 7.8749e-14)

ans = 

  フィールドをもつ struct:

    yalmiptime: 0.0937
    solvertime: 0.1733
          info: 'Numerical problems (SeDuMi-1.3)'
       problem: 4

Nonlinear scalar (real, models 'norm', 1 variable, current value : 0.1861)

ans = 

  フィールドをもつ struct:

    yalmiptime: 0.1038
    solvertime: 0.0672
          info: 'Successfully solved (SeDuMi-1.3)'
       problem: 0

Nonlinear scalar (real, models 'norm', 1 variable, current value : 2.3714e-11)

ans = 

  フィールドをもつ struct:

    yalmiptime: 0.1059
    solvertime: 0.1851
          info: 'Numerical problems (SeDuMi-1.3)'
       problem: 4

Nonlinear scalar (real, models 'norm', 1 variable, current value : 0.00095883)

ans = 

  フィールドをもつ struct:

    yalmiptime: 0.0904
    solvertime: 0.1146
          info: 'Numerical problems

Quadratic matrix variable 2x2 (symmetric, real, 3 variables, eigenvalues between [1.125,5.872195685431699e+16])
Linear matrix variable 4x1 (full, real, 3 variables, values in range [-7621483127462159,5.771552037616335e+16])

